In [ ]:
# Cell-1 — Load Model and Level-3 Dataset
import os, sys
import pandas as pd
from typing import Any, Dict, List
import ast

# Deterministic repo root discovery
def find_repo_root(start_dir=None):
    d = start_dir or os.getcwd()
    while True:
        if os.path.exists(os.path.join(d, 'requirements.txt')) or os.path.exists(os.path.join(d, '.git')):
            return d
        parent = os.path.dirname(d)
        if parent == d:
            return os.getcwd()
        d = parent
repo_root = find_repo_root()
sys.path.insert(0, repo_root)

# Canonical intents (do not change)
INTENTS = ['investigate','execute','summarize','ops']

# Attempt to locate a Level-2 inference model deterministically
l2_model = None
model_name = None
candidates = [
    ('level2.level2_model','Level2Classifier'),
    ('level2.model','Level2Classifier'),
    ('level0.level0_model','Level0Classifier')
]

for mod, cls in candidates:
    try:
        m = __import__(mod, fromlist=[cls])
        ModelClass = getattr(m, cls)
        try:
            if hasattr(ModelClass, 'load'):
                for try_path in [os.path.join(repo_root, 'level2', 'models'), os.path.join(repo_root, 'level0', 'models'), os.path.join(repo_root, mod.split('.')[0], 'models')]:
                    if os.path.exists(try_path):
                        try:
                            l2_model = ModelClass.load(try_path)
                            break
                        except Exception:
                            continue
            if l2_model is None:
                try:
                    l2_model = ModelClass()
                except Exception:
                    l2_model = None
        except Exception:
            l2_model = None
        if l2_model is not None:
            model_name = f'{mod}.{cls}'
            break
    except Exception:
        continue

if l2_model is None:
    raise RuntimeError('No Level-2 inference model found')

# Locate Level-3 dataset
candidates_paths = [os.path.join(repo_root, 'l3', 'data', 'level3_intents.csv'), os.path.join(repo_root, 'level3', 'data', 'level3_intents.csv')]
level3_path = None
for p in candidates_paths:
    if os.path.exists(p):
        level3_path = p
        break
if level3_path is None:
    raise FileNotFoundError('level3/data/level3_intents.csv not found')

l3_df = pd.read_csv(level3_path)
required = ['utterance','gold_intent','allowed_intents','suppressed_intents']
missing = [c for c in required if c not in l3_df.columns]
if missing:
    raise ValueError(f'Missing required Level-3 columns: {missing}')

def _parse_list_cell(x):
    if isinstance(x, (list, tuple)):
        return list(x)
    if pd.isna(x):
        return []
    s = str(x).strip()
    if s == '':
        return []
    try:
        v = ast.literal_eval(s)
        if isinstance(v, (list, tuple)):
            return list(v)
    except Exception:
        pass
    return [item.strip() for item in s.split(',') if item.strip()]

for c in ['allowed_intents','suppressed_intents']:
    l3_df[c] = l3_df[c].apply(_parse_list_cell)

L2_MODEL = l2_model
MODEL_NAME = model_name
L3_DF = l3_df
INTENT_ORDER = INTENTS
print(f'Loaded model: {MODEL_NAME}')
print(f'Loaded {len(L3_DF)} records from Level-3 dataset')

In [ ]:
# Cell-2 — Baseline Level-2 Inference (Unconstrained)
from typing import Dict

def predict_probs(utterance: str) -> Dict[str, float]:
    try:
        if hasattr(L2_MODEL, 'predict_proba') and hasattr(L2_MODEL, 'classes_'):
            probs = L2_MODEL.predict_proba([utterance])[0]
            classes = list(L2_MODEL.classes_)
            mapping = {c: float(probs[i]) for i,c in enumerate(classes)}
            return {k: float(mapping.get(k, 0.0)) for k in INTENT_ORDER}
    except Exception:
        pass
    try:
        out = L2_MODEL.predict(utterance)
        if isinstance(out, dict) and 'probabilities' in out:
            probs = out['probabilities']
            return {k: float(probs.get(k, 0.0)) for k in INTENT_ORDER}
        if isinstance(out, dict) and 'intent' in out and 'confidence' in out:
            p = {k: 0.0 for k in INTENT_ORDER}
            if out['intent'] in p:
                p[out['intent']] = float(out['confidence'])
            return p
    except Exception:
        pass
    try:
        out = L2_MODEL.predict_proba(utterance)
        if isinstance(out, dict):
            return {k: float(out.get(k, 0.0)) for k in INTENT_ORDER}
    except Exception:
        pass
    try:
        lab = None
        pred = L2_MODEL.predict(utterance)
        if isinstance(pred, dict) and 'intent' in pred:
            lab = pred['intent']
        elif isinstance(pred, str):
            lab = pred
        p = {k: 0.0 for k in INTENT_ORDER}
        if lab in p:
            p[lab] = 1.0
        return p
    except Exception:
        raise RuntimeError('Unable to obtain model predictions')

rows = []
for _, r in L3_DF.iterrows():
    utt = r['utterance']
    probs = predict_probs(utt)
    s = sum(probs.values())
    if s <= 0:
        probs = {k: 1.0/len(INTENT_ORDER) for k in INTENT_ORDER}
        s = 1.0
    if abs(s - 1.0) > 1e-12:
        probs = {k: float(v)/float(s) for k,v in probs.items()}
    top_intent = max(probs.items(), key=lambda iv: (iv[1], iv[0]))[0]
    top_score = probs[top_intent]
    rows.append({
        'utterance': utt,
        'gold_intent': r['gold_intent'],
        'raw_probs': probs,
        'raw_top_intent': top_intent,
        'raw_top_score': float(top_score),
        'allowed_intents': r['allowed_intents'],
        'suppressed_intents': r['suppressed_intents']
    })
BASELINE_DF = pd.DataFrame(rows)
L2_BASELINE = BASELINE_DF
print(f'Completed baseline inference on {len(L2_BASELINE)} utterances')

In [ ]:
# Cell-3 — Apply Level-2.5 Logical Constraints (Inference-Time Only)
from typing import Any, Dict

def apply_constraints_row(row) -> Dict[str, Any]:
    probs = dict(row['raw_probs'])
    allowed = [a.strip().lower() for a in (row.get('allowed_intents') or [])]
    suppressed = [s.strip().lower() for s in (row.get('suppressed_intents') or [])]
    for s in suppressed:
        if s in probs:
            probs[s] = 0.0
    if len(allowed) > 0:
        for k in list(probs.keys()):
            if k not in allowed:
                probs[k] = 0.0
    total = sum(probs.values())
    constrained_flag = True
    if total <= 0.0:
        probs = dict(row['raw_probs'])
        constrained_flag = False
        total = sum(probs.values())
    if abs(total - 1.0) > 1e-12:
        probs = {k: float(v)/float(total) for k,v in probs.items()}
    top_intent = max(probs.items(), key=lambda iv: (iv[1], iv[0]))[0]
    top_score = probs[top_intent]
    return {
        'constrained_probs': probs,
        'constrained_top_intent': top_intent,
        'constrained_top_score': float(top_score),
        'constrained_applied': constrained_flag
    }

rows = []
for _, r in L2_BASELINE.iterrows():
    rows.append(apply_constraints_row(r))
CONSTRAINED_DF = pd.concat([L2_BASELINE.reset_index(drop=True), pd.DataFrame(rows)], axis=1)
L2_2P5 = CONSTRAINED_DF
print(f'Applied Level-2.5 constraints to {len(L2_2P5)} predictions')

In [ ]:
# Cell-4 — Comparative Metrics (Before vs After)
def is_violating_raw(row):
    ai = [a.strip().lower() for a in (row.get('allowed_intents') or [])]
    si = [s.strip().lower() for s in (row.get('suppressed_intents') or [])]
    rt = str(row['raw_top_intent']).strip().lower()
    if rt in si:
        return True
    if len(ai) > 0 and rt not in ai:
        return True
    return False

def is_violating_constrained(row):
    ai = [a.strip().lower() for a in (row.get('allowed_intents') or [])]
    si = [s.strip().lower() for s in (row.get('suppressed_intents') or [])]
    ct = str(row['constrained_top_intent']).strip().lower()
    if ct in si:
        return True
    if len(ai) > 0 and ct not in ai:
        return True
    return False

df = L2_2P5.copy()
df['viol_raw'] = df.apply(is_violating_raw, axis=1)
df['viol_constrained'] = df.apply(is_violating_constrained, axis=1)
total = len(df)
viol_before = int(df['viol_raw'].sum())
viol_after = int(df['viol_constrained'].sum())
pct_before = viol_before/total*100 if total else 0.0
pct_after = viol_after/total*100 if total else 0.0
flips = (df['raw_top_intent'] != df['constrained_top_intent']).sum()
flip_rate = int(flips)/total*100 if total else 0.0
df['delta_top_score'] = df['constrained_top_score'] - df['raw_top_score']
mean_delta = float(df['delta_top_score'].mean())
dist_before = df['raw_top_intent'].value_counts().to_dict()
dist_after = df['constrained_top_intent'].value_counts().to_dict()
print('Comparative Metrics (Before vs After)')
print(f'Total records: {total}')
print(f'% violating before: {pct_before:.2f}% ({viol_before})')
print(f'% violating after : {pct_after:.2f}% ({viol_after})')
print(f'Intent flip rate: {flip_rate:.2f}% ({flips})')
print(f'Mean change in top-score (after - before): {mean_delta:.4f}')
print('\nDistribution before (intent: count)')
for k in INTENTS:
    print(f'  {k}: {int(dist_before.get(k,0))}')
print('\nDistribution after (intent: count)')
for k in INTENTS:
    print(f'  {k}: {int(dist_after.get(k,0))}')

# Save violation columns back to L2_2P5 for downstream cells
L2_2P5 = df

L25_METRICS = {
    'total': total,
    'viol_before': viol_before,
    'viol_after': viol_after,
    'pct_before': pct_before,
    'pct_after': pct_after,
    'flip_count': int(flips),
    'flip_rate_pct': flip_rate,
    'mean_delta_top_score': mean_delta,
    'dist_before': dist_before,
    'dist_after': dist_after
}

In [ ]:
# Cell-5 — Concrete Example Review
candidates = L2_2P5[L2_2P5['viol_raw'] == True].copy()
corrected = candidates[(candidates['viol_constrained'] == False) & (candidates['raw_top_intent'] != candidates['constrained_top_intent'])]
corrected = corrected.sort_values(by='utterance')
sample = corrected.head(10)
print('Concrete examples where L2 was invalid and L2.5 corrected:')
for _, r in sample.iterrows():
    print('\n---')
    print('utterance :', r['utterance'])
    print('raw L2   :', r['raw_top_intent'])
    print('constrained L2.5 :', r['constrained_top_intent'])
    print('suppressed intents :', r['suppressed_intents'])

L25_EXAMPLES = sample

In [ ]:
# Cell-6 — Level-2.5 Verdict
before = L25_METRICS['viol_before']
after = L25_METRICS['viol_after']
reduction = before - after
reduction_pct = (reduction / before) if before else 0.0
if reduction_pct > 0.10:
    verdict = 'LEVEL-2.5 SHOWS CLEAR VALUE'
else:
    verdict = 'LEVEL-2.5 SHOWS LIMITED VALUE'
if reduction_pct > 0.10:
    justification = 'Constraint application reduced logical violations by more than 10% — Level-3 modeling may be justified.'
else:
    justification = 'Constraint application produced limited reduction in violations — further Level-3 modeling should be evaluated conservatively.'
print(verdict)
print(f'Violations before: {before}, after: {after}, reduction: {reduction} ({reduction_pct:.2%})')
print(justification)
L25_VERDICT = {'verdict': verdict, 'before': before, 'after': after, 'reduction': reduction, 'reduction_pct': reduction_pct}

# What did Level-2.5 prove?

## 1. What kinds of errors Level-2 makes

Level-2 models (TF-IDF, embeddings, or neural classifiers) produce predictions based purely on statistical patterns in training data. This structural design creates characteristic error modes:

- **Logical contradictions**: The model can assign high probability to mutually exclusive intents (e.g., predicting both `execute` and `investigate` for the same utterance, when investigation should precede action).

- **Overconfident but invalid predictions**: When an utterance matches training examples strongly, the model confidently predicts intents that violate domain rules (e.g., executing an action that the user's role forbids).

- **Ambiguous cases where multiple intents compete**: Without explicit logic, the model has no principled way to resolve conflicts between valid but contradictory interpretations.

- **Errors caused by lack of structural constraints**: The model learns correlations but not causality. It cannot distinguish "utterances that happened to co-occur with intent X" from "utterances that must produce intent X given the domain logic."

**This is not a data quality problem or a model weakness problem.** These errors arise because statistical models fundamentally lack mechanisms to enforce logical constraints. The model architecture itself has no space to represent rules like "if suppressed, probability must be zero."

---

## 2. Which errors are logically impossible vs. undesirable

Not all errors are equal. NSAI systems distinguish:

### Logically impossible errors
These violate explicit domain rules and should **never** occur:

- **Violations of explicit domain rules**: Predicting intents that are explicitly suppressed for a given utterance
- **Mutually exclusive intent combinations**: Assigning probability to intents that cannot co-exist by definition
- **Actions that should never occur given the utterance form**: For example, executing a command when the utterance is phrased as a question

These errors are **hard constraints** — they represent logical contradictions, not just poor choices.

### Logically undesirable errors
These are valid under the rules but suboptimal:

- **Valid but suboptimal intent selection**: Choosing `summarize` when `investigate` would yield better outcomes
- **Poor prioritization under uncertainty**: Assigning equal weight to all allowed intents when context suggests one is more appropriate
- **Premature action vs. investigation**: Immediately executing when gathering information first would be safer

These errors represent **soft preferences** — the model made a legal move, but not the best one.

**This distinction is the foundation of neuro-symbolic AI.** Hard constraints belong in logic. Soft preferences belong in learned models.

---

## 3. How Level-2.5 fixes these without retraining

Level-2.5 applies logical constraints **after** the Level-2 model produces its predictions:

1. **The model predicts freely**: Level-2 generates a full probability distribution over all intents, using only statistical patterns.

2. **Logic filters invalid outputs**: For each prediction, Level-2.5 sets the probability of suppressed intents to zero. If `allowed_intents` is specified, it zeros all intents not in that list.

3. **Remaining probabilities are re-normalized**: After removing invalid options, the remaining probabilities are scaled to sum to 1.0.

4. **The model itself is unchanged**: No weights are updated. No retraining occurs. The model's internal representations remain identical.

**The model still thinks freely — logic only corrects the output.** The Level-2 model may internally prefer invalid paths, but those preferences are masked before the user sees them. This is efficient and interpretable, but it comes with fundamental limitations.

---

## 4. What Level-2.5 cannot fix

Level-2.5 proves that post-hoc logic helps, but it also reveals why post-hoc logic is not enough:

- **The model can still internally prefer invalid paths**: Even though we zero out suppressed intents, the model's embeddings and learned weights still encode those invalid patterns. The model wastes representational capacity on options it will never select.

- **Conflicts are resolved late**: By the time logic intervenes, the model has already committed to a prediction. If the top-ranked intent is invalid, we fall back to the second choice — but that second choice was learned without knowledge that the first was impossible.

- **Representation learning is unchanged**: The model's hidden layers continue to encode features that correlate with suppressed intents. These features pollute the representation space and interfere with learning valid patterns.

- **Logic cannot shape how the model reasons**: Constraints that apply during inference cannot influence how the model learns to decompose the problem. If certain intent combinations are impossible, the model should learn features that reflect that structure — but Level-2.5 cannot teach it to do so.

**This is why Level-3 exists.** To fix these limitations, logic must be embedded **inside** the model architecture, not applied as a post-processing filter.

---

## 5. Key takeaway

**Level-2.5 proves that logical constraints reduce invalid predictions.** In cases where the base model frequently violates domain rules, inference-time filtering can deliver immediate value without retraining.

**Level-2.5 also proves that post-hoc logic has fundamental limits.** The model learns and reasons without awareness of constraints, leading to wasted capacity and suboptimal representations.

**True neuro-symbolic reasoning requires logic inside the model** — not as a filter, but as a structural component of learning and inference. That is the transition from Level-2.5 to Level-3.